This requires you to write a 2D GAN game. I let you to get into the topic yourself, whitout any explonations from my side. You can watch lecture, seminar, read papers and tutorials (fun, fun, fun).

### Homework

I want you to implement a simple 2D GAN game. The kind of animation, I want to see is like in [this video](https://www.youtube.com/watch?v=KeJINHjyzOU) at 15:30 or in [here](https://habrahabr.ru/post/275429/) but in 2D. You can google, search code at github, whatever, but the network should be based on Theano. 

Basically you will need to come up with true distribution $P$, say mixture of gaussians (surprise me), sample some data from it. Visualize it as a heatmap. To visualize $G$ density you can fix $N$ noise vectors $\{z_i\} \quad i=1,\dots, N$ and draw a circle for each $G(z_i)$. It is also funny to visualize discriminator as a vector field (can be done with `plt.arrow`, `plt.quiver`). Look how it should be in the middle of [this page](http://www.inference.vc/an-alternative-update-rule-for-generative-adversarial-networks/).

Please, render your animation to video (`.mp4`) or to `.webm` format. `.gif` typically is very heavy in size.

And make sure your code works if 'Run All' is pressed and it draws the animation you've rendered.

Good luck!

In [62]:
import matplotlib
matplotlib.use('Agg')
import numpy as np
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

import theano
import theano.tensor as T
from lasagne.nonlinearities import rectify, sigmoid, linear, tanh, elu

from lasagne.layers import DenseLayer, InputLayer, get_output
import lasagne

from scipy.stats import multivariate_normal

#%matplotlib inline

In [63]:
#define true P(no surprizes) and plot it 
np.random.seed = 239
mu1, cov1, mu2, cov2 = (2,3), [[3, -0.2], [-0.2, 3]], (6,5), [[2, 0.7], [0.7, 2]]
true_pdf = lambda x : multivariate_normal.pdf(x, mu1, cov1)# + multivariate_normal.pdf(x, mu2, cov2)
true_rvs = lambda size : multivariate_normal.rvs(mu1, cov1, size)# + multivariate_normal.rvs(mu2, cov2, size)

delta = 0.1
x = np.arange(0., 10.0, delta)
y = np.arange(0., 10.0, delta)
X, Y = np.meshgrid(x, y)
Z = true_pdf(np.vstack((X.reshape(-1), Y.reshape(-1))).T).reshape(X.shape)

CS = plt.contour(X, Y, Z, 50)
plt.show()

In [127]:
G_input = T.matrix('Gx')
G_l1 = InputLayer((None, 2), G_input)
G_l2 = DenseLayer(G_l1, 30, nonlinearity=rectify)
G_l3 = DenseLayer(G_l2, 30, nonlinearity=rectify)
G_l4 = DenseLayer(G_l3, 30, nonlinearity=tanh)
G_l5 = DenseLayer(G_l4, 10, nonlinearity=tanh)
G_l6 = DenseLayer(G_l5, 2, nonlinearity=linear)
G = G_l6

G_out = get_output(G)

# discriminators
D1_input = T.matrix('D1x')
D1_l1 = InputLayer((None, 2), D1_input)
D1_l2 = DenseLayer(D1_l1, 10, nonlinearity=tanh)
D1_l3 = DenseLayer(D1_l2, 10, nonlinearity=tanh)
#D1_l4 = DenseLayer(D1_l3, 10, nonlinearity=tanh)
D1_l5 = DenseLayer(D1_l3, 1, nonlinearity=sigmoid)
D1 = D1_l5

D2_l1 = InputLayer((None, 2), G_out)
D2_l2 = DenseLayer(D2_l1, 10, nonlinearity=tanh, W=D1_l2.W, b=D1_l2.b)
D2_l3 = DenseLayer(D2_l2, 10, nonlinearity=tanh, W=D1_l3.W, b=D1_l3.b)
#D2_l4 = DenseLayer(D2_l3, 10, nonlinearity=sigmoid, W=D1_l4.W, b=D1_l4.b)
D2_l5 = DenseLayer(D2_l3, 1, nonlinearity=sigmoid, W=D1_l5.W, b=D1_l5.b)
D2 = D2_l5

D1_out = get_output(D1)
D2_out = get_output(D2)

In [128]:
# objectives
G_obj = (T.log(D2_out)).mean()
D_obj = (T.log(D1_out) + T.log(1 - D2_out)).mean()

# parameters update and training
G_params = lasagne.layers.get_all_params(G, trainable=True)
G_lr = theano.shared(np.array(0.01, dtype=theano.config.floatX))
G_updates = lasagne.updates.nesterov_momentum(1 - G_obj, G_params, learning_rate=G_lr, momentum=0.6)
#G_updates = lasagne.updates.adam(1-G_obj, G_params, learning_rate=G_lr)
G_train = theano.function([G_input], G_obj, updates=G_updates)

D_params = lasagne.layers.get_all_params(D1, trainable=True)
D_lr = theano.shared(np.array(0.1, dtype=theano.config.floatX))
D_updates = lasagne.updates.nesterov_momentum(1 - D_obj, D_params, learning_rate=D_lr, momentum=0.6)
#D_updates = lasagne.updates.adam(1-D_obj, D_params, learning_rate=D_lr)
D_train = theano.function([G_input, D1_input], D_obj, updates=D_updates)


In [129]:
G_function = theano.function([G_input], G_out)

In [130]:
D_function = theano.function([D1_input],D1_out)

In [131]:
sample_noise = lambda size : np.vstack(((np.random.random(size) * 10), np.random.random(size)*10))

In [136]:
random_sample = sample_noise(100)

In [137]:
delta = 0.1
x = np.arange(0., 10.0, delta)
y = np.arange(0., 10.0, delta)
X, Y = np.meshgrid(x, y)

delta = 0.1
x_d = np.linspace(-10., 20.0, 100)
y_d = np.linspace(-10., 10.0, 100)
X_d, Y_d = np.meshgrid(x_d, y_d)


def plot_all(generator, descriminator):
    f = plt.figure()
    canvas = matplotlib.backends.backend_agg.FigureCanvas(f)
    subplot = f.add_subplot(111)
    Z = true_pdf(np.vstack((X.reshape(-1), Y.reshape(-1))).T).reshape(X.shape)
    subplot.contour(X,Y,Z,20)

    Z = descriminator(np.vstack((X_d.reshape(-1), Y_d.reshape(-1))).T).reshape(X.shape)
    CS = plt.contour(X_d,Y_d,Z,np.linspace(0, 1,15), colors ='r', )
    subplot.clabel(CS , inline=1, fontsize=5)

    rs = generator(random_sample.reshape(100,2))
    subplot.scatter(rs[:,0], rs[:,1])

    canvas.draw()
    shape = canvas.get_width_height()
    frame = np.fromstring(canvas.tostring_argb(), dtype='uint8').reshape(shape[1], shape[0], 4)

    del f
    return frame

In [138]:
test_image = plot_all(G_function, D_function)

In [139]:
default_shape = test_image.shape
video_shape = np.zeros(4)
video_shape[:3] = default_shape

Actually i was not able to achive smth good tuning learning rate, rate of training for descr and gen etc. This current version is strange, because discriminator fails (g_obj = -0.695, d(g(x)) = 0.5), although the generator is not that good. 
Adding one layer to descr makes it much better than generator and it quickly wins.

In [176]:
epochs = 2000
video_shape[3] = epochs
frames = np.zeros(video_shape.astype(int))

batch_size = 20
M = 200  # mini-batch size

f = matplotlib.figure.Figure()
graph = f.add_subplot(111)
    
for i in range(epochs):
    for j in range(10):
        x = true_rvs(200)  # sampled orginal batch
        z = sample_noise(200)
        d = D_train(z.reshape(200,2), x.reshape(200,2))
    z = sample_noise(200)
    g =  G_train(z.reshape(200, 2))
    if i % 10 == 0:  # lr decay
        G_lr *= 0.99
        D_lr *= 0.99
        
    frames[:,:,:,i] = plot_all(G_function,D_function)
        
    if i % 20 == 0 :
        print i, g, d
        #plot_all(G_function,D_function)
        #curr_p = D_function(np.concatenate((X.reshape(100,100,1),Y.reshape(100,100,1)), axis=2).reshape(10000,2)).reshape(100,100)
#        CS = plt.contour(X, Y, curr_p, 20)
#        curr_points = G_function(random_sample.reshape(1000,2))
#        plt.scatter(curr_points[:,0], curr_points[:, 1])
#        plt.show()

0 -0.699076470406 -1.379983087
20 -0.694169404908 -1.38485758538
40 -0.697058407014 -1.38460753879
60 -0.699582963819 -1.37809389318
80 -0.702073632756 -1.37414080276


In [163]:
import pylab
import imageio

In [177]:
vid = imageio.get_writer("./asasa_no_one_wins.mp4", 'ffmpeg')

In [178]:
for i in range(0,epochs,10):

    vid.append_data(frames[:,:,:,i])

In [179]:
vid.close()

In [180]:
G_lr.eval()

array(0.0023794816241159552)

In [181]:
D_lr.eval()

array(0.023794816241159554)